<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/(%EB%8C%80%EB%B6%84%EB%A5%98)_%EB%AA%A8%EB%91%90%EB%8B%A4_%EC%A6%9D%EB%B6%84%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%82%AC%EC%9A%A9_%EC%9D%B4%EA%B2%83%EC%9D%B4_%EC%B5%9C%EC%A2%85%EC%9E%84_model_33%EC%9C%BC%EB%A1%9C_%EC%A0%80%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 168 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 13.9 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595744 sha256=7d53ceae01015c2a8597f5f029a451ae09f9855fa31bd8128065193e1257c9eb
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 14.1 MB/s 
     |████████████████████████████████| 769 kB 14.7 MB/s 
     |████████████████████████████████| 3.0 MB 89.5 MB/s 
     |████████████████████████████████| 895 kB 88.1 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-qu75b9nv
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 데이터		   '답안 작성용 파일.csv'     labels.zip
 산업분류자동화     dabunru_inc_model_22.pt  'My Drive'
'Colab Notebooks'   dabunru_inc_model.pt      val.txt


In [ ]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'모델 개발용 자료'.zip -d /content/dataset
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/모델 개발용 자료.zip
2. 모델개발용자료.txt:  mismatching "local" filename (2. ыкиыН╕ъ░Ьы░ЬьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/2. 모델개발용자료.txt  
Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [ ]:
import pandas as pd
df_A=pd.read_csv('/content/gdrive/MyDrive/데이터/df_A_inc.csv') #증분데이터 사용
df_B=pd.read_csv('/content/gdrive/MyDrive/데이터/df_B_inc.csv') #증분데이터 사용
df_C=pd.read_csv('/content/gdrive/MyDrive/데이터/df_C_inc.csv')
df_D=pd.read_csv('/content/gdrive/MyDrive/데이터/df_D_inc.csv') #증분데이터 사용
df_E=pd.read_csv('/content/gdrive/MyDrive/데이터/df_E_inc.csv') #증분데이터 사용
df_F=pd.read_csv('/content/gdrive/MyDrive/데이터/df_F_inc_dain.csv')
df_G=pd.read_csv('/content/gdrive/MyDrive/데이터/df_G_inc.csv')
df_H=pd.read_csv('/content/gdrive/MyDrive/데이터/spell_check/H_spellcheck.csv')
df_I=pd.read_csv('/content/gdrive/MyDrive/데이터/spell_check/I_spell_check.csv')
df_J=pd.read_csv('/content/gdrive/MyDrive/데이터/df_J_inc.csv') #증분데이터 사용
df_K=pd.read_csv('/content/gdrive/MyDrive/데이터/df_K_inc.csv') #증분데이터 사용
df_L=pd.read_csv('/content/gdrive/MyDrive/데이터/df_L_inc.csv')
df_M=pd.read_csv('/content/gdrive/MyDrive/데이터/df_M_inc.csv')
df_N=pd.read_csv('/content/gdrive/MyDrive/데이터/df_N_inc.csv') #증분데이터 사용
df_O=pd.read_csv('/content/gdrive/MyDrive/데이터/df_O_inc.csv') #증분데이터 사용
df_P=pd.read_csv('/content/gdrive/MyDrive/데이터/df_P_inc.csv')
df_Q=pd.read_csv('/content/gdrive/MyDrive/데이터/df_Q_inc.csv')
df_R=pd.read_csv('/content/gdrive/MyDrive/데이터/df_R_inc.csv')
df_S=pd.read_csv('/content/gdrive/MyDrive/데이터/spell_check/S_spellcheck.csv')

In [ ]:
df=pd.concat([df_A, df_B, df_C, df_D, df_E, df_F, df_G, df_H, df_I, df_J, df_K, df_L, df_M, df_N, df_O, df_P, df_Q, df_R, df_S])

In [ ]:
df['digit_1'].value_counts()

C    830871
G    352042
I    187425
P    107494
S    100396
H     98038
Q     86657
M     74308
F     73466
L     67852
K     54098
R     52271
N     27296
J     22852
A     13275
E     10428
O      6380
B      5847
D      1968
Name: digit_1, dtype: int64

**데이터 전처리**

In [ ]:
df.loc[(df['digit_1'] == "A"), 'digit_1'] = 0  
df.loc[(df['digit_1'] == "B"), 'digit_1'] = 1  
df.loc[(df['digit_1'] == "C"), 'digit_1'] = 2  
df.loc[(df['digit_1'] == "D"), 'digit_1'] = 3  
df.loc[(df['digit_1'] == "E"), 'digit_1'] = 4  
df.loc[(df['digit_1'] == "F"), 'digit_1'] = 5  
df.loc[(df['digit_1'] == "G"), 'digit_1'] = 6  
df.loc[(df['digit_1'] == "H"), 'digit_1'] = 7  
df.loc[(df['digit_1'] == "I"), 'digit_1'] = 8  
df.loc[(df['digit_1'] == "J"), 'digit_1'] = 9  
df.loc[(df['digit_1'] == "K"), 'digit_1'] = 10  
df.loc[(df['digit_1'] == "L"), 'digit_1'] = 11 
df.loc[(df['digit_1'] == "M"), 'digit_1'] = 12  
df.loc[(df['digit_1'] == "N"), 'digit_1'] = 13  
df.loc[(df['digit_1'] == "O"), 'digit_1'] = 14  
df.loc[(df['digit_1'] == "P"), 'digit_1'] = 15  
df.loc[(df['digit_1'] == "Q"), 'digit_1'] = 16 
df.loc[(df['digit_1'] == "R"), 'digit_1'] = 17  
df.loc[(df['digit_1'] == "S"), 'digit_1'] = 18  

In [ ]:
data_list = []
for q, label in zip(df['text'], df['digit_1'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['벼 맥류 콩류 잡곡류 종자 생산 보급', '0'], ['벼 모판 생산 농가 출하', '0'], ['사업장에서 오이. 고추를 재배 오이. 고추를 판매', '0'], ['볍씨 발아 모종', '0'], ['  벼 재배', '0']]


In [ ]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

Kobert 입력데이터 만들기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

Kobert 학습 모델 만들기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=19,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [18]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/30558 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.9376182556152344 train acc 0.09375
epoch 1 batch id 201 loss 2.77531099319458 train acc 0.08916355721393035
epoch 1 batch id 401 loss 2.1832354068756104 train acc 0.22334788029925187
epoch 1 batch id 601 loss 2.016319990158081 train acc 0.3014767054908486
epoch 1 batch id 801 loss 1.641176700592041 train acc 0.378647784019975
epoch 1 batch id 1001 loss 1.3717889785766602 train acc 0.4456481018981019
epoch 1 batch id 1201 loss 1.00736403465271 train acc 0.5019645087427144
epoch 1 batch id 1401 loss 0.6920409202575684 train acc 0.5493955210563883
epoch 1 batch id 1601 loss 0.6061652898788452 train acc 0.589885227982511
epoch 1 batch id 1801 loss 0.5141634345054626 train acc 0.6226922543031649
epoch 1 batch id 2001 loss 0.6015062928199768 train acc 0.6502686156921539
epoch 1 batch id 2201 loss 0.3925855755805969 train acc 0.6742602794184461
epoch 1 batch id 2401 loss 0.3327088952064514 train acc 0.6948927530195752
epoch 1 batch id 2601 loss 0.35961949825286865 tr

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/3396 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9847993746968753


  0%|          | 0/30558 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2609051764011383 train acc 0.96875
epoch 5 batch id 201 loss 0.004693792201578617 train acc 0.9916044776119403
epoch 5 batch id 401 loss 0.0024091133382171392 train acc 0.9911549251870324
epoch 5 batch id 601 loss 0.038612544536590576 train acc 0.9910045757071547
epoch 5 batch id 801 loss 0.013016393408179283 train acc 0.990753745318352
epoch 5 batch id 1001 loss 0.008754552341997623 train acc 0.9907436313686314
epoch 5 batch id 1201 loss 0.011029865592718124 train acc 0.9906458159866778
epoch 5 batch id 1401 loss 0.0023982145357877016 train acc 0.9905870806566738
epoch 5 batch id 1601 loss 0.12306080758571625 train acc 0.9904942223610244
epoch 5 batch id 1801 loss 0.0011191981611773372 train acc 0.9902831760133259
epoch 5 batch id 2001 loss 0.06218138709664345 train acc 0.9901767866066966
epoch 5 batch id 2201 loss 0.13085384666919708 train acc 0.9900613357564744
epoch 5 batch id 2401 loss 0.04076730087399483 train acc 0.9901538421491045
epoch 5 batch id 2601

  0%|          | 0/3396 [00:00<?, ?it/s]

epoch 5 test acc 0.9850570307628352


a,b,d,e,o만 증분데이터 acc 0.982008457

In [19]:
#학습 모델 저장
torch.save(model, 'dabunryu_inc_model_33.pt')

In [20]:
model_save_name='dabunru_inc_model_33.pt'
path=F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model.state_dict(), path)